In [1]:
## https://docs.sqlalchemy.org/en/14/orm/quickstart.html

In [2]:
from sqlalchemy import Column
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship

In [3]:
Base = declarative_base()

In [4]:
from model.user import User
from model.address import Address

class User(Base):
     __tablename__ = "user_account"

     id = Column(Integer, primary_key=True)
     name = Column(String(30))
     fullname = Column(String)

     addresses = relationship(
         "Address", back_populates="user", cascade="all, delete-orphan"
     )

     def __repr__(self):
         return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
     __tablename__ = "address"

     id = Column(Integer, primary_key=True)
     email_address = Column(String, nullable=False)
     user_id = Column(Integer, ForeignKey("user_account.id"), nullable=False)

     user = relationship("User", back_populates="addresses")

     def __repr__(self):
         return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [5]:
from sqlalchemy import create_engine
engine = create_engine("sqlite://", echo=True, future=True)

In [6]:
Base.metadata.create_all(engine)

2022-06-01 18:44:32,128 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-01 18:44:32,129 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
from sqlalchemy.orm import Session

with Session(engine) as session:

     spongebob = User(
         name="spongebob",
         fullname="Spongebob Squarepants",
         addresses=[Address(email_address="spongebob@sqlalchemy.org")],
     )
     sandy = User(
         name="sandy",
         fullname="Sandy Cheeks",
         addresses=[
             Address(email_address="sandy@sqlalchemy.org"),
             Address(email_address="sandy@squirrelpower.org"),
         ],
     )
     patrick = User(name="patrick", fullname="Patrick Star")

     session.add_all([spongebob, sandy, patrick])

     session.commit()
        

InvalidRequestError: When initializing mapper mapped class Address->address, expression 'User' failed to locate a name ('User'). If this is a class name, consider adding this relationship() to the <class 'model.address.Address'> class after both dependent classes have been defined.

In [ ]:
from sqlalchemy import select

session = Session(engine)

stmt = select(User).where(User.name.in_(["spongebob", "sandy"]))

for user in session.scalars(stmt):
     print(user)